In [1]:
import astropy.time
from lsst.daf.butler import Butler, Timespan

In [2]:
butler = Butler('/repo/embargo')
registry = butler.registry

In [3]:
scratch_butler = Butler('/home/j/jarugula/scratch', writeable=True)
scratch_registry = scratch_butler.registry

In [4]:
#help(scratch_registry)

In [58]:
datasetType = 'raw'
collections = 'LATISS/raw/all'
instrument = 'LATISS'
embargohours = 1 #hour
#nowtime = 'now'
nowtime = '2022-10-05 18:37:52.232000'

In [30]:
dataId = {'instrument': instrument}

In [59]:
embargo_period = astropy.time.TimeDelta(embargohours*3600., format='sec')
if nowtime != 'now':
    now = astropy.time.Time(nowtime, scale='tai', format='iso')
else:
    now = astropy.time.Time.now()
timespan_embargo = Timespan(now - embargo_period, now)

In [60]:
now - embargo_period

<Time object: scale='tai' format='iso' value=2022-10-05 17:37:52.232>

In [61]:
timespan_embargo

Timespan(begin=astropy.time.Time('2022-10-05 17:37:52.232000', scale='tai', format='iso'), end=astropy.time.Time('2022-10-05 18:37:52.232000', scale='tai', format='iso'))

In [65]:
after_embargo = []
#after_embargo_2 = []

for i, dt in enumerate(registry.queryDimensionRecords('exposure', dataId=dataId, datasets=datasetType,
                                                          collections=collections,
                                                          #where="NOT exposure.timespan OVERLAPS timespan_embargo",
                                                          where="exposure.timespan OVERLAPS timespan_embargo",
                                                          bind={"timespan_embargo": timespan_embargo})):
        #print(dt.id, dt.timespan)
        after_embargo.append(dt.id)
        # end_time = dt.timespan.end
        # if now - end_time <= embargo_period:
        #     print(dt.id, dt.timespan)
        #     after_embargo_2.append(dt.id)
        #     print('--------------')

In [66]:
datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections=collections,
                                         where="exposure.id IN (exposure_ids)",
                                         bind={"exposure_ids": after_embargo})

In [68]:
scratch_butler.transfer_from(butler, source_refs=datasetRefs, transfer='copy',
                           skip_missing=True, register_dataset_types=True,
                           transfer_dimensions=True)

[DatasetRef(DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure), {instrument: 'LATISS', detector: 0, exposure: 2022100500221, ...}, id=0af1dfa3-6d7b-5378-bec4-17f9350d5f05, run='LATISS/raw/all'),
 DatasetRef(DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure), {instrument: 'LATISS', detector: 0, exposure: 2022100500230, ...}, id=826be40c-b63e-5f06-bc41-47208ce70f0b, run='LATISS/raw/all'),
 DatasetRef(DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure), {instrument: 'LATISS', detector: 0, exposure: 2022100500210, ...}, id=4f963757-d661-50c3-895f-d9d166fc3df9, run='LATISS/raw/all'),
 DatasetRef(DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure), {instrument: 'LATISS', detector: 0, exposure: 2022100500231, ...}, id=863222c2-7fd7-55b5-8f31-de9a6e447fb7, run='LATISS/raw/all'),
 DatasetRef(DatasetType('raw', {band, instrument, detector, physical_filter, exp

In [69]:
scratch_butler.pruneCollection(collections, purge=True, unstore=True)